In [2]:
import pandas as pd
import codecs as cd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline


## Loading Delek data

In [3]:
path = 'C:/Users/yehoshua/Documents/Master of Engineering - Business Analytics/Thesis - DriveWize/Data samples/data samples 20160131/fuel_01_31_12_15.csv'
reader = cd.open(path, 'r', encoding = 'utf-16')
newD1 = []
for line in reader:
    x = line.split('\t')
    newD1.append(x)
reader.close()

In [4]:
D2 = pd.DataFrame(data = newD1[1:], columns=['trip_id', 'car_id','car_group','driver_name','driver_group', 'employee_num', 'start_time'
                                             , 'start_loc','end_time','end_loc', 'distance','duration','fuel_cons','km_per_liter'
                                             ,'liter_per_100km','total_distance','NaN'])
D2["new_start_time"] = pd.to_datetime(D2.start_time, format = "%d/%m/%Y %H:%M:%S")
D2["new_end_time"] = pd.to_datetime(D2.end_time, format = "%d/%m/%Y %H:%M:%S")
D2['start_time'],D2['start_date']= D2['new_start_time'].apply(lambda x:x.time()), D2['new_start_time'].apply(lambda x:x.date())
D2['end_time'],D2['end_date']= D2['new_end_time'].apply(lambda x:x.time()), D2['new_end_time'].apply(lambda x:x.date())
DelekData = D2.drop(['NaN', 'new_start_time','new_end_time'], axis=1)
DelekData['duration'] = pd.to_timedelta(DelekData.duration)
DelekData['fuel_cons'].replace('', np.nan, inplace=True)
DelekData.dropna(subset=['fuel_cons'], inplace=True)
DelekData['fuel_cons'] = DelekData['fuel_cons'].convert_objects(convert_numeric=True)

In [5]:
MinimizedDelekData = pd.DataFrame(data = DelekData, columns =['car_id', 'start_date', 'duration','fuel_cons'])

In [6]:
groupedDelek = MinimizedDelekData.groupby(['car_id','start_date'], as_index=False)

def get_hours(td):
    ans = float(td.seconds)/3600
    if ans == 0:
        return -0.000001
    return ans 

def avg_fuel(group):
    f = group['fuel_cons']
    d = group['duration']
    ans = f.sum() / get_hours(d.sum()) , get_hours(d.sum())
    return ans

groupedDelek = groupedDelek.apply(avg_fuel)
g1 = groupedDelek.reset_index()

g1['avg_fuel'] , g1['total_duration_hr'] = g1.iloc[:,2].apply(lambda x:x[0]), g1.iloc[:,2].apply(lambda x:x[1])
g1 = g1.drop(g1.columns[2], axis=1)

In [7]:
g1.sort(['avg_fuel'])

,car_id,start_date,avg_fuel,total_duration_hr
982,90-295-30,2015-12-22,0.302521,1.983333
942,90-293-30,2015-12-15,0.305284,8.516667
932,90-293-30,2015-12-02,0.817635,8.316667
893,90-291-30,2015-12-17,1.430233,5.733333
817,72-696-52,2015-12-27,1.490683,2.683333
973,90-295-30,2015-12-09,1.565217,1.150000
818,72-696-52,2015-12-28,1.764706,0.566667
986,90-295-30,2015-12-26,2.057143,0.583333
821,72-696-52,2015-12-31,2.185185,2.700000
592,63-829-70,2015-12-14,2.198020,1.683333


## Loading ISR data

In [8]:
pathISR = 'C:/Users/yehoshua/Documents/Master of Engineering - Business Analytics/Thesis - DriveWize/Data samples/data samples 20160131/aggressive_01_12_12_15.csv'
pathISR2 = 'C:/Users/yehoshua/Documents/Master of Engineering - Business Analytics/Thesis - DriveWize/Data samples/data samples 20160131/aggressive_13_25_12_15.csv'
pathISR3 = 'C:/Users/yehoshua/Documents/Master of Engineering - Business Analytics/Thesis - DriveWize/Data samples/data samples 20160131/aggressive_26_31_12_15.csv'

reader = cd.open(pathISR, 'r', encoding = 'utf-16')
reader2 = cd.open(pathISR2, 'r', encoding = 'utf-16')
reader3 = cd.open(pathISR3, 'r', encoding = 'utf-16')

reader.next() #skip the header
reader2.next()
reader3.next()

newD1 = []
for line in reader:
    x = line.split('\t')
    newD1.append(x)
for line in reader2:
    x = line.split('\t')
    newD1.append(x)
for line in reader3:
    x = line.split('\t')
    newD1.append(x)

reader.close()
reader2.close()
reader3.close()

In [9]:
D3 = pd.DataFrame(data = newD1[1:], columns = ['driver_name','driver_group','car_id', 'car_group','event_time','location','event_type','event_category','event_end_time','end_location','distance','velocity','max_speed','duration','comment','NaN'])
D3["new_event_time"] = pd.to_datetime(D3.event_time, format = "%d/%m/%Y %H:%M:%S")
D3['event_time'],D3['event_date']= D3['new_event_time'].apply(lambda x:x.time()), D3['new_event_time'].apply(lambda x:x.date())
ISRData = D3.drop(['NaN', 'new_event_time'], axis=1)

In [10]:
ISRData

,driver_name,driver_group,car_id,car_group,event_time,location,event_type,event_category,event_end_time,end_location,distance,velocity,max_speed,duration,comment,event_date
0,NA,,63-380-70,מטרופולין השרון,22:38:03,התדהר אזור תעשיהרעננה,בטיחותי,פניה בינונית ימינה,,,,24,50,,,2015-12-02
1,NA,,63-380-70,מטרופולין השרון,22:38:52,התדהר אזור תעשיהרעננה,בטיחותי,בלימה חזקה,,,,1,50,,,2015-12-02
2,NA,,63-380-70,מטרופולין השרון,22:40:12,התדהר רעננה,בטיחותי,פניה בינונית שמאלה,,,,32,50,,,2015-12-02
3,NA,,63-380-70,מטרופולין השרון,22:41:04,כביש 4 כפר סבא,בטיחותי,פניה בינונית שמאלה,,,,31,90,,,2015-12-02
4,NA,,63-380-70,מטרופולין השרון,23:20:26,חלמיש פארק העסקים והתעשיה קיסריה,בטיחותי,פניה בינונית שמאלה,,,,28,50,,,2015-12-02
5,NA,,63-380-70,מטרופולין השרון,12:35:15,גרניט 2פארק העסקים והתעשיה קיסריה,בטיחותי,פניה בינונית שמאלה,,,,25,50,,,2015-12-06
6,NA,,63-380-70,מטרופולין השרון,12:56:35,הכורם פארק העסקים והתעשיה קיסריה,בטיחותי,פניה בינונית שמאלה,,,,28,50,,,2015-12-06
7,NA,,63-380-70,מטרופולין השרון,12:56:49,הכורם פארק העסקים והתעשיה קיסריה,בטיחותי,פניה בינונית שמאלה,,,,25,50,,,2015-12-06
8,NA,,63-380-70,מטרופולין השרון,12:57:31,גרניט פארק העסקים והתעשיה קיסריה,בטיחותי,פניה בינונית שמאלה,,,,33,50,,,2015-12-06
9,NA,,63-380-70,מטרופולין השרון,12:58:30,תרשיש 8פארק העסקים והתעשיה קיסריה,בטיחותי,פניה בינונית שמאלה,,,,26,50,,,2015-12-06


In [11]:
MinimizedISRData = pd.DataFrame(data = ISRData, columns =['car_id', 'event_date', 'event_category'])

def get_event_weight(row):
    #print row
    if 'מאוד' in row['event_category'].encode('utf8'):
        return 1
    elif 'בינוני' in row['event_category'].encode('utf8'):
        return 1
    return 1

MinimizedISRData['weight'] = MinimizedISRData.apply(get_event_weight, axis=1)

groupedISR = MinimizedISRData.groupby(['car_id' , 'event_date'], as_index=False)

def get_event_count(group):
    a = group['weight']
    return a.sum()

groupedISR = groupedISR.apply(get_event_count)
#groupedISR = groupedISR.apply(get_event_weight)

g2 = groupedISR.reset_index()

new_columns = g2.columns.values
#new_columns[2] = 'Category'
new_columns[2] = 'Total_events'
g2.columns = new_columns

## Join the two tables

In [13]:
joined = pd.merge(g1[(g1.start_date < '2015-12-13')], g2, left_on =['car_id', 'start_date'], right_on= ['car_id','event_date'], how = 'left')
joined['Total_events'] = joined['Total_events'].replace(to_replace='NaN', value = 0)
def avg_events(x):    
    return x['Total_events']/ x['total_duration_hr']

joined['avg_events']= joined.apply(avg_events, axis=1)

In [27]:
joined.iloc[:,2:].head()

,avg_fuel,total_duration_hr,event_date,Total_events,avg_events
1,8.030769,1.083333,2015-12-02,5,4.615385
4,8.350962,6.933333,2015-12-06,29,4.182692
6,8.045455,13.200000,2015-12-08,110,8.333333
7,8.052632,9.500000,2015-12-09,40,4.210526
9,8.635606,13.200000,2015-12-01,90,6.818182


#### Spliting Train- Test datasets

In [15]:
joined = joined[(joined['avg_fuel'] < 11)]
joined = joined[(joined['avg_fuel'] > 5)]
joined = joined[(joined['total_duration_hr'] > 1)]

from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(joined.drop('avg_fuel', axis=1), joined['avg_fuel'], test_size=0.2, random_state=0)

## Regression model

In [16]:
results = sm.OLS(y_train,X_train['avg_events']).fit()

print results.summary()

                            OLS Regression Results                            
Dep. Variable:               avg_fuel   R-squared:                       0.782
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     873.4
Date:                Sat, 20 Feb 2016   Prob (F-statistic):           2.01e-82
Time:                        21:35:10   Log-Likelihood:                -678.51
No. Observations:                 244   AIC:                             1359.
Df Residuals:                     243   BIC:                             1363.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
avg_events     1.2281      0.042     29.553      0.0

#### MSE on train data

In [17]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_train, results.predict(X_train['avg_events']))

15.236118156969304

#### MSE on test data

In [18]:
print mean_squared_error(y_test, results.predict(X_test['avg_events']))

17.9894420223
